In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '/Users/alperkose/Desktop/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
def split_data(x, y, ratio,seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO
    
    
    c = list(zip(x, y))

    np.random.shuffle(c)

    x, y = zip(*c)
    
    ind=round(ratio*len(x))
    ind2=len(x)
    x=np.array(x)
    trainx = x[0:ind-1,:]
    trainy = y[0:ind-1]
    testx = x[ind:ind2-1,:]
    testy = y[ind:ind2-1]
    
    return trainx, testx,trainy,testy
    # ***************************************************
    raise NotImplementedError

In [4]:
tX,testx,y,testy=split_data(tX, y, 0.8,seed=1)

In [5]:
tX[tX==-999]=0
m=np.mean(tX,axis=0)
centered_tX=tX-m
centered_tX[centered_tX==-m]=0


standardized_tX=centered_tX / np.std(centered_tX, axis=0)

d=len(standardized_tX[0])
n=len(standardized_tX)
mat=np.zeros((n,int(d*(d+1)/2+d)))
q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_tX[i,t]

        
for i in range (n):
    for monsoon in range (d):
        for t in range (monsoon,d): 
            mat[i,tao]=standardized_tX[i,t]*standardized_tX[i,q]
            tao=tao+1
        q=q+1    
    tao=d  
    q=0

    



In [6]:
num_samples=len(mat)
tx = np.c_[np.ones(num_samples), mat]

In [7]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # least squares: TODO
    # returns mse, and optimal weights
    xtx=np.dot(tx.transpose(),tx)
    xy=np.dot(tx.transpose(),y)
    w=np.dot(np.linalg.inv(xtx),xy)
    #e=y-np.dot(tx,w)
    #mse=np.dot(e.transpose(),e)/(2*len(tx))
    return w
    # ***************************************************

In [8]:
def ridge_regression(y, tx, lamb):
    """implement ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    xtx=np.dot(tx.transpose(),tx)
    l_inside=2*lamb*len(tx)*np.eye(tx.shape[1])
    ins=xtx+l_inside
    xy=np.dot(tx.transpose(),y)
    
    w=np.dot(np.linalg.inv(ins),xy)
    e=y-np.dot(tx,w)
    mse=np.dot(e.transpose(),e)/(2*len(tx))
    
    return w
    # ***************************************************
    raise NotImplementedError

In [9]:
def compute_loss(y, tx, w):
    """Calculate the loss.
    
    You can calculate the loss using mse or mae.
    """
    # ***************************************************
    e=y-np.dot(tx,w)
    mse=np.dot(e.transpose(),e)/(2*len(tx))
    return mse
    # ***************************************************
    raise NotImplementedError

In [10]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    # ***************************************************
    e=y-np.dot(tx,w)
    return (-1/len(tx))*np.dot(tx.transpose(),e)
    # ***************************************************
    raise NotImplementedError

In [11]:
def least_squares_GD(y, tx, gamma, max_iters):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    initial_w=np.zeros(tx.shape[1])
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: compute gradient and loss
        grad=compute_gradient(y,tx,w)
        loss=compute_loss(y,tx,w)
        # ***************************************************
        gamma=gamma/1.005
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: update w by gradient
        w=w-gamma*grad
        # ***************************************************
        
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

In [12]:
from plots import gradient_descent_visualization
import datetime
# Define the parameters of the algorithm.
max_iters = 100
gamma = 0.0002

# Initialization
#w_initial = np.array([0.0, 0.0])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = least_squares_GD(y, tx, gamma, max_iters)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

Gradient Descent(0/99): loss=0.5, w0=-6.28112593299285e-05, w1=-1.068336422568577e-05
Gradient Descent(1/99): loss=0.4982547749351945, w0=-0.00012503951243584744, w1=-2.0763507914137518e-05
Gradient Descent(2/99): loss=0.496595316790688, w0=-0.00018670535384794415, w1=-3.0266843521689202e-05
Gradient Descent(3/99): loss=0.49501318489875906, w0=-0.00024782536563590243, w1=-3.921761504412473e-05
Gradient Descent(4/99): loss=0.4935025167838092, w0=-0.00030841363238479254, w1=-4.763843034319548e-05
Gradient Descent(5/99): loss=0.492058457624324, w0=-0.0003684825674529154, w1=-5.5550569442658576e-05
Gradient Descent(6/99): loss=0.4906766938390041, w0=-0.0004280433793418558, w1=-6.297417456191304e-05
Gradient Descent(7/99): loss=0.4893532963011151, w0=-0.0004871063476367299, w1=-6.99283764883267e-05
Gradient Descent(8/99): loss=0.4880846533351708, w0=-0.0005456809963989478, w1=-7.643138569033479e-05
Gradient Descent(9/99): loss=0.4868674318005186, w0=-0.0006037762109362137, w1=-8.25005630902

In [13]:
least_squares_w=least_squares(y,tx)

In [14]:
lamb=0.00075
ridge_w=ridge_regression(y, tx, lamb)

In [15]:
testx[testx==-999]=0
m2=np.mean(testx,axis=0)
centered_testx=testx-m2
centered_testx[centered_testx==-m2]=0
standardized_testx=centered_testx / np.std(centered_testx, axis=0)


d=len(standardized_testx[0])
n=len(standardized_testx)
mat=np.zeros((n,int(d*(d+1)/2+d)))
q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_testx[i,t]

        
for i in range (n):
    for mk in range (d):
        for t in range (mk,d): 
            mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,q]
            tao=tao+1
        q=q+1    
    tao=d  
    q=0

    
num_samples=len(standardized_testx)
final_testx = np.c_[np.ones(num_samples), mat]


In [16]:
w=ridge_w
#w=least_squares_w
#w=gradient_ws[-1]
aq=np.dot(final_testx,w)
y_guess=np.sign(aq)
trueness=sum((y_guess==testy))/len(testy)
print(trueness)

0.79897597952


## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '/Users/alperkose/Desktop/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
testx=tX_test
testx[testx==-999]=0
m2=np.mean(testx,axis=0)
centered_testx=testx-m2
centered_testx[centered_testx==-m2]=0
standardized_testx=centered_testx / np.std(centered_testx, axis=0)



d=len(standardized_testx[0])
n=len(standardized_testx)
mat=np.zeros((n,d*(d+1)/2+d))
q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_testx[i,t]

        
for i in range (n):
    for m in range (d):
        for t in range (m,d): 
            mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,q]
            tao=tao+1
        q=q+1    
    tao=d  
    q=0


num_samples=len(standardized_testx)
final_testx = np.c_[np.ones(num_samples), mat]
tX_test=final_testx
weights=ridge_w
#tX_test = np.c_[np.ones(len(tX_test)), tX_test]
#weights=gradient_ws[-1]
#w=gradient_ws[-1]
#aq=np.dot(tX_test,w)
#y_guess=np.sign(aq)
#trueness=sum((y_guess==y))/y.shape[0]
#print(trueness)

In [ ]:
OUTPUT_PATH = '/Users/alperkose/Desktop/deneme8.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)